# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [48]:
#include <stdio.h>

int main(void) {
    float S = 200.0;
    int N = 5;
    int J = 1;
    float parcela = S;
    for (int mes = 1; mes <= N; mes++) {
        if (mes > 1)
            parcela *= 1 + (J / 100.0);
        printf("Parcela #%d = %.2f\n", mes, parcela);
    }
    return 0;
}

Parcela #1 = 200.00
Parcela #2 = 202.00
Parcela #3 = 204.02
Parcela #4 = 206.06
Parcela #5 = 208.12


## Exercício Parte 2 - Escrevendo um Módulo

Reescreva o código acima de forma que seu programa faça uso de uma função que seja responsável pelo cálculo de uma parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal.

In [50]:
#include <stdio.h>

/* Calcula a enésima parcela, partindo toda vez da parcela inicial. */
float calcular_parcela(float parcela_anterior, int juros, int numero_da_parcela);

int main(void) {
    float S = 200.0;
    int N = 5;
    int J = 1;
    float parcela_inicial = S;
    for (int mes = 1; mes <= N; mes++)
        printf("Parcela #%d = %.2f\n", mes, calcular_parcela(S, J, mes));
    
    return 0;
}

float calcular_parcela(float parcela_inicial, int juros, int numero_da_parcela) {
    float parcela_resultante = parcela_inicial;
    for (int mes = 1; mes <= numero_da_parcela; mes++)
        if (mes > 1)
            parcela_resultante *= 1 + (juros / 100.0);
    
    return parcela_resultante;
}

Parcela #1 = 200.00
Parcela #2 = 202.00
Parcela #3 = 204.02
Parcela #4 = 206.06
Parcela #5 = 208.12


## Exercício Parte 3 - Minimizando os Parâmetros

Modifique o módulo para que o cliente (que chama a função) use o mínimo de parâmetros possíveis. Para isso a sua função precisa estar ciente do estado corrente do empréstimo, ou seja, ela precisará saber qual a próxima parcela que ela precisa calcular sem que lhe seja informado, para isso:
* evite informar recorrentemente como parâmetros dados sobre as características do empréstimo;
* evite usar informações de controle (e.g., parcela corrente) como parâmetro.

In [51]:
#include <stdio.h>
#include <stdbool.h>

/* Calcula a próxima parcela sem utilizar parâmetros, empregando variáveis globais. */
float proxima_parcela();

float S = 200.0;
int N = 5;
int J = 1;
bool primeiro_pagamento = true;

int main(void) {
    for (int mes = 1; mes <= N; mes++)
        printf("Parcela #%d = %.2f\n", mes, proxima_parcela());
    
    return 0;
}

float proxima_parcela() {
    if (primeiro_pagamento)
        primeiro_pagamento = false;
    else
        S *= (1 + (J / 100.0));
    return S;
}

Parcela #1 = 200.00
Parcela #2 = 202.00
Parcela #3 = 204.02
Parcela #4 = 206.06
Parcela #5 = 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Partindo da Solução da Parte 2, modifique a forma como você representa o empréstimo, de forma que a função que calcula a próxima parcela receba um único parâmetro.

In [52]:
#include <stdio.h>
#include <stdbool.h>

/* Implementa a dívida na forma de uma estrutura */
typedef struct Debt {
    float parcela_atual;
    int qtd_remanescente;
    int juros;
    bool primeiro_pagamento;
} Debt;

/* Inicializa um objeto Debt com três parâmetros do usuário */
Debt set_debt(float parcela_atual, int qtd_remanescente, int juros);
/* Elimina uma das parcelas remanescentes e calcula o valor da próxima. */
Debt pagar_parcela(Debt divida);

int main(void) {
    Debt divida = set_debt(200.0, 5, 1);
    int mes = 1;
    while (divida.qtd_remanescente > 0) {
        divida = pagar_parcela(divida);
        printf("Parcela #%d = %.2f\n", mes, divida.parcela_atual);
        mes++;
    }
    
    return 0;
}

Debt set_debt(float parcela_atual, int qtd_remanescente, int juros) {
    Debt divida;
    divida.parcela_atual = parcela_atual;
    divida.qtd_remanescente = qtd_remanescente;
    divida.juros = juros;
    divida.primeiro_pagamento = true;
    
    return divida;
}

Debt pagar_parcela(Debt divida) {
    divida.qtd_remanescente -= 1;
    if (divida.primeiro_pagamento)
        divida.primeiro_pagamento = false;
    else
        divida.parcela_atual *= 1 + (divida.juros / 100.0);
    
    return divida;
}

Parcela #1 = 200.00
Parcela #2 = 202.00
Parcela #3 = 204.02
Parcela #4 = 206.06
Parcela #5 = 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. O seu programa deve ser capaz de informar as parcelas de mais de um empréstimo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar o programa principal, a função ou ambos para atender esse requisito da melhor forma possível.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [54]:
#include <stdio.h>
#include <stdbool.h>
#include <stdlib.h>

/* Implementa a dívida na forma de uma lista ligada */
typedef struct _Debt {
    float parcela_atual;
    int qtd_remanescente;
    int juros;
    bool primeiro_pagamento;
    bool quitada;
    struct _Debt *prox;
} Debt;

/* Inicializa um objeto Debt com três parâmetros do usuário, retornando ponteiro */
Debt *set_debt(float parcela_atual, int qtd_remanescente, int juros);
/* Para cada dívida na lista ligada, decresce a quantia de parcelas remanescentes e calcula o valor da próxima */
void pagar_lista_parcelas(Debt *divida);

int main(void) {
    Debt *divida1 = set_debt(200.0, 5, 1);
    (*divida1).prox = set_debt(500.0, 7, 2);
    int num_emprestimo = 1;
    int num_parcela = 1;
    bool falta_quitar = true;
    Debt *ptr_divida;
    while (falta_quitar) {
        pagar_lista_parcelas(divida1);
        ptr_divida = divida1;
        falta_quitar = false;
        while (ptr_divida != NULL) {
            if ( !(*ptr_divida).quitada ) {
                printf("Emprestimo %d: parcela %d eh %.2f\n", num_emprestimo, num_parcela, (*ptr_divida).parcela_atual);
                if ((*ptr_divida).qtd_remanescente == 0)
                    (*ptr_divida).quitada = true;
                else
                    falta_quitar = true;
            }
            ptr_divida = (*ptr_divida).prox;
            num_emprestimo++;
        }
        num_parcela++;
        num_emprestimo = 1;
    }
    
    return 0;
}

Debt *set_debt(float parcela_atual, int qtd_remanescente, int juros) {
    Debt *divida = malloc(sizeof(Debt));
    (*divida).parcela_atual = parcela_atual;
    (*divida).qtd_remanescente = qtd_remanescente;
    (*divida).juros = juros;
    (*divida).primeiro_pagamento = true;
    (*divida).quitada = false;
    (*divida).prox = NULL;
    
    return divida;
}

void pagar_lista_parcelas(Debt *divida) {
    Debt *ptr_divida = divida;
    while (ptr_divida != NULL) {
        if ( !(*ptr_divida).quitada ) {
            (*ptr_divida).qtd_remanescente -= 1;
            if ((*ptr_divida).primeiro_pagamento)
                (*ptr_divida).primeiro_pagamento = false;
            else
                (*ptr_divida).parcela_atual *= 1 + ((*ptr_divida).juros / 100.0);
        }
        ptr_divida = (*ptr_divida).prox;
    }
    
    return;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
